In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import torch.optim as optim
from planet.models.determinstic_state import DeterministicStateModel
from planet.models.stochastic_state import StochasticStateModel
from planet.models.reward import RewardModel
from planet.models.encoder import ImageEncoderModel
from planet.models.observation import ImageObservationModel
from planet.trainer import train

from planet.utils.wrappers import RepeatActionWrapper, GymPixelWrapper, ImagePreprocessorWrapper
from planet.utils.seed import set_seed


%load_ext autoreload
%autoreload 2

In [ ]:
# set seed for reproducibility
set_seed(0)

In [ ]:
# initialize the environment
env = GymPixelWrapper(
    domain_name="cheetah", task_name="run", render_kwargs={'width': 64, 'height': 64}
)
env = RepeatActionWrapper(env, skip=4)
env = ImagePreprocessorWrapper(env)

In [ ]:
# action info
action = env.action_space.sample()
action_size = action.shape[0]

# model sizes
state_size = 30
hidden_state_size = 200
observation_size = 1024
hidden_layer_size = 300

In [ ]:
det_state_model = DeterministicStateModel(
    hidden_state_size=hidden_state_size,
    state_size=state_size,
    action_size=action_size,
    hidden_layer_size=hidden_layer_size
).cuda()

In [ ]:
stoch_state_model = StochasticStateModel(
    hidden_state_size=hidden_state_size,
    state_size=state_size,
    hidden_layer_size=hidden_layer_size,
).cuda()

In [ ]:
obs_model = ImageObservationModel(
    hidden_state_size=hidden_state_size,
    state_size=state_size,
    observation_size=observation_size,
).cuda()

In [ ]:
reward_obs_model = RewardModel(
    hidden_state_size=hidden_state_size,
    state_size=state_size,
    hidden_layer_size=hidden_layer_size,
).cuda()

In [ ]:
enc_model = ImageEncoderModel(
    hidden_state_size=hidden_state_size,
    observation_size=observation_size,
    state_size=state_size,
    hidden_layer_size=hidden_layer_size,
).cuda()

In [ ]:
models = {
    "det_state_model": det_state_model,
    "stoch_state_model": stoch_state_model,
    "obs_model": obs_model,
    "reward_obs_model": reward_obs_model,
    "enc_model": enc_model,
}


lr = 1e-3
eps = 1e-4
optimizers = {
    "all_params": optim.Adam(
        list(det_state_model.parameters()) + 
        list(stoch_state_model.parameters()) + 
        list(obs_model.parameters()) + 
        list(reward_obs_model.parameters()) + 
        list(enc_model.parameters()), 
        lr=lr, 
        eps=eps
    ),
}



In [11]:
train(
    env=env,
    train_steps=10_000,
    T=250,
    S=5,
    C=100,
    B=50,
    L=50,
    H=12,
    I=10,
    J=1000,
    K=100,
    models=models,
    optimizers=optimizers,
    hidden_state_size=hidden_state_size,
    state_size=state_size,
    action_size=action_size,
    log_interval=1,
    evaluate_interval=10,
    action_noise=0.1,
    free_nats=3.0,
)